<a href="https://colab.research.google.com/github/Chitkala19/restaurant_review/blob/main/Restaurent_Review_Sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libraries
import numpy as np
import pandas as pd
# from google.colab import drive
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [2]:
data_set = pd.read_csv('/content/dataset_2.csv',encoding='latin-1')


In [3]:


# Print first 5 rows
print("Top 5 rows of dataset")
print(data_set.describe())

# Print random 10 rows
print("Random 10 rows")
print(data_set.sample(10))

# Print total number of rows and columns in dataset
print("Total rows and columns in dataset")
print(data_set.shape)

# Print number of rows and columns in dataset separately
print("Total Rows =", data_set.shape[0])
print("Total Columns =", data_set.shape[1])

print("Columns in dataset")
print(data_set.columns)

#Check information of dataset
print("Dataset information")
print(data_set.info())


Top 5 rows of dataset
           review sentiment
count       35398     35398
unique      23274         4
top     Excellent  positive
freq         2166     18437
Random 10 rows
                                                  review sentiment
30987                                          Excellent  positive
6365               Very quick service but not very clean  negative
7656                                                Poor  negative
13237  Very bad service. Do not give receipt. Do not ...  negative
34641  (It wasn't busy either) Also, the building was...  negative
1489                                           Excellent  positive
30403  Busy but orders are right. Needs someone on th...  positive
32283        Got me waiting for to long to take my order   neutral
28127                   Don't like to order on a machine  negative
11835                              Quick breakfast place   neutral
Total rows and columns in dataset
(35398, 2)
Total Rows = 35398
Total Columns = 2
Colu

In [4]:
data_set.describe()

data_set.head()

,review,sentiment
0,Why does it look like someone spit on my food?...,positive
1,It'd McDonalds. It is what it is as far as the...,positive
2,Made a mobile order got to the speaker and che...,negative
3,My mc. Crispy chicken sandwich was Ã¯Â¿Â½Ã¯Â¿Â...,neutral
4,"I repeat my order 3 times in the drive thru, a...",negative


In [5]:

#Check for duplicate values
print("Checking for duplicate data")
print("Total Duplicated values =", data_set.duplicated().sum())

#Check for null values
print("Checking for null values")
print("Total NULL values =\n\n",data_set.isnull().sum())

print("Sort by no of ratings")
print(data_set['sentiment'].value_counts())

Checking for duplicate data
Total Duplicated values = 11763
Checking for null values
Total NULL values =

 review       0
sentiment    0
dtype: int64
Sort by no of ratings
positive     18437
negative     10321
neutral       6639
sentiment        1
Name: sentiment, dtype: int64


In [6]:
stopwords_set = set(stopwords.words('english'))
def processing(text):

    # Step 1: Remove Accented Characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Step 2: Tokenization
    tokens = word_tokenize(text)

    # Step 3: Stopwords Removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Step 4: Remove Numbers and Extra Whitespaces
    filtered_tokens = [re.sub(r'\d+', '', word) for word in filtered_tokens]
    filtered_tokens = [word.strip() for word in filtered_tokens if word.strip()]

    # Step 5: Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Step 6: Stemming (optional)
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in lemmatized_tokens]

    # Step 7: Remove Single Letters
    filtered_tokens = [word for word in stemmed_tokens if len(word) > 1]

    # Join the tokens back into a clean text string
    clean_text = ' '.join(filtered_tokens)


    return clean_text

In [7]:
i=0
for reviews in data_set['review']:
  data_set.at[i,'review'] = processing(reviews)
  # data_set.at[i,'rating']= data_set.at[i,'rating'][0]
  # data_set.at[i,'review_time'] = convert_time_to_days(data_set.at[i,'review_time'])
  # print(convert_time_to_days(data_set.at[i,'review_time']))
  # if i==5:
  #   break
  i+=1
data_set.head()

random_sample = data_set.sample(n=20, random_state=42)
random_sample

sia = SentimentIntensityAnalyzer()
sentiments = []

In [8]:
for review in data_set['review']:
    sentiment = sia.polarity_scores(review)
    sentiments.append(sentiment)

i=0
while i!=10:
  print(sentiments[i])
  i+=1

sentiment_labels = []

for sentiment in sentiments:
    compound_score = sentiment['compound']
    # compound_score = sentiment
    if compound_score >= 0.05:
        sentiment_labels.append(1)
    elif compound_score <= -0.05:
        sentiment_labels.append(-1)
    else:
        sentiment_labels.append(0)

{'neg': 0.047, 'neu': 0.769, 'pos': 0.184, 'compound': 0.5541}
{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'compound': 0.7003}
{'neg': 0.032, 'neu': 0.968, 'pos': 0.0, 'compound': -0.25}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.2, 'neu': 0.8, 'pos': 0.0, 'compound': -0.7184}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.025, 'neu': 0.951, 'pos': 0.023, 'compound': -0.0423}
{'neg': 0.029, 'neu': 0.84, 'pos': 0.13, 'compound': 0.7003}
{'neg': 0.125, 'neu': 0.875, 'pos': 0.0, 'compound': -0.25}
{'neg': 0.024, 'neu': 0.64, 'pos': 0.336, 'compound': 0.9759}


In [9]:
data_set['sentiment'] = sentiment_labels
data_set[['review', 'sentiment']]

X = data_set['review']
y = data_set['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [10]:
# Create a LinearSVC model
linear_svc_model = LinearSVC()

# Fit the model to the TF-IDF training data
linear_svc_model.fit(X_train_tfidf, y_train)

# Predict on the TF-IDF test data
linear_svc_y_pred = linear_svc_model.predict(X_test_tfidf)

# Calculate accuracy and print the results
linear_svc_accuracy = accuracy_score(y_test, linear_svc_y_pred)
print("LinearSVC Accuracy:", linear_svc_accuracy)
print("Classification Report:")
print(classification_report(y_test, linear_svc_y_pred))

# Create a MultinomialNB model
naive_bayes_model = MultinomialNB()

# Fit the model to the TF-IDF training data
naive_bayes_model.fit(X_train_tfidf, y_train)

# Predict on the TF-IDF test data
naive_bayes_y_pred = naive_bayes_model.predict(X_test_tfidf)

LinearSVC Accuracy: 0.928954802259887
Classification Report:
              precision    recall  f1-score   support

          -1       0.89      0.83      0.86      1441
           0       0.92      0.96      0.94      1958
           1       0.95      0.95      0.95      3681

    accuracy                           0.93      7080
   macro avg       0.92      0.91      0.92      7080
weighted avg       0.93      0.93      0.93      7080



In [11]:

# Calculate accuracy and print the results
naive_bayes_accuracy = accuracy_score(y_test, naive_bayes_y_pred)
print("Multinomial Naive Bayes Accuracy:", naive_bayes_accuracy)
print("Classification Report:")
print(classification_report(y_test, naive_bayes_y_pred))

Multinomial Naive Bayes Accuracy: 0.7283898305084746
Classification Report:
              precision    recall  f1-score   support

          -1       0.86      0.54      0.66      1441
           0       0.95      0.39      0.56      1958
           1       0.67      0.98      0.80      3681

    accuracy                           0.73      7080
   macro avg       0.83      0.64      0.67      7080
weighted avg       0.79      0.73      0.70      7080



In [12]:
# Create a Logistic Regression model
logistic_regression_model = LogisticRegression()

# Fit the model to the TF-IDF training data
logistic_regression_model.fit(X_train_tfidf, y_train)

# Predict on the TF-IDF test data
logistic_regression_y_pred = logistic_regression_model.predict(X_test_tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# Calculate accuracy and print the results
logistic_regression_accuracy = accuracy_score(y_test, logistic_regression_y_pred)
print("Logistic Regression Accuracy:", logistic_regression_accuracy)
print("Classification Report:")
print(classification_report(y_test, logistic_regression_y_pred))

Logistic Regression Accuracy: 0.9088983050847458
Classification Report:
              precision    recall  f1-score   support

          -1       0.87      0.76      0.81      1441
           0       0.87      0.95      0.91      1958
           1       0.94      0.94      0.94      3681

    accuracy                           0.91      7080
   macro avg       0.90      0.89      0.89      7080
weighted avg       0.91      0.91      0.91      7080



In [14]:
# Create a Random Forest model
random_forest_model = RandomForestClassifier()

# Fit the model to the TF-IDF training data
random_forest_model.fit(X_train_tfidf, y_train)

# Predict on the TF-IDF test data
random_forest_y_pred = random_forest_model.predict(X_test_tfidf)

In [15]:
# Calculate accuracy and print the results
random_forest_accuracy = accuracy_score(y_test, random_forest_y_pred)
print("Random Forest Accuracy:", random_forest_accuracy)
print("Classification Report:")
print(classification_report(y_test, random_forest_y_pred))

model1 = LinearSVC()
model2 = MultinomialNB()
model3 = LogisticRegression()

ensemble_model = VotingClassifier(
    estimators=[('model1', model1), ('model2', model2), ('model3', model3)],
    voting='hard'  # You can choose 'hard' or 'soft' voting depending on your use case
)

Random Forest Accuracy: 0.9269774011299435
Classification Report:
              precision    recall  f1-score   support

          -1       0.92      0.81      0.86      1441
           0       0.90      0.96      0.93      1958
           1       0.94      0.96      0.95      3681

    accuracy                           0.93      7080
   macro avg       0.92      0.91      0.91      7080
weighted avg       0.93      0.93      0.93      7080



In [16]:


# Fit the ensemble model on the training data
ensemble_model.fit(X_train_tfidf, y_train)

# Make predictions using the ensemble model
y_pred = ensemble_model.predict(X_test_tfidf)

# Evaluate the performance of the ensemble model (you can use different metrics)
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Model Accuracy: {accuracy:.2f}')

# Finally, you can use the trained ensemble model to make predictions on your test data
y_test_pred = ensemble_model.predict(X_test_tfidf)

# Save the ensemble model
joblib.dump(ensemble_model, 'ensemble_model_1.joblib')
joblib.dump(vectorizer, 'tfidf_vectorizer_1.joblib')

loaded_model = joblib.load('ensemble_model_1.joblib')
loaded_vectorizer = joblib.load('tfidf_vectorizer_1.joblib')
Input="The delectable aroma of the freshly baked bread filled the air as I stepped into the cozy bakery. The warm, crusty baguette I ordered was a delightful treat. It had a perfect crunch on the outside and a soft, airy interior. Absolutely delicious!"
input_vector = loaded_vectorizer.transform([processing(Input)])
predicted_rating = loaded_model.predict(input_vector)
print("Predicted Rating:", predicted_rating)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Model Accuracy: 0.93
Predicted Rating: [1]
